In [1]:
!pip install openai

In [2]:
# Import the libraries
import pandas as pd
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [3]:
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [4]:
if openai.api_key.startswith('sk'):
    print('sucessfully loaded OPENAI_API_KEY')

sucessfully loaded OPENAI_API_KEY


# Exploratory Data Analysis

In [14]:
df = pd.read_csv('laptop_data.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [20]:
df.columns

Index(['Brand', 'Model Name', 'Core', 'CPU Manufacturer', 'Clock Speed',
       'RAM Size', 'Storage Type', 'Display Type', 'Display Size',
       'Graphics Processor', 'Screen Resolution', 'OS', 'Laptop Weight',
       'Special Features', 'Warranty', 'Average Battery Life', 'Price',
       'Description'],
      dtype='object')

### Convert to Numeric

In [15]:
import pandas as pd
import re

def convert_to_numeric(df, column_name):
    """
    Converts a given column in the DataFrame to numeric values by stripping non-numeric characters.
    Adds a new column with the processed numeric values.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to convert.

    Returns:
        pd.DataFrame: The updated DataFrame with a new numeric column.
    """
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Strip non-numeric characters using regular expressions
    def strip_and_convert(value):
        # Remove all non-numeric characters, except for the decimal point
        numeric_value = re.sub(r'[^\d.]', '', str(value))
        try:
            return float(numeric_value) if numeric_value else None
        except ValueError:
            return None

    # Create the new column with numeric values
    new_column_name = f"{column_name} (numeric)"
    df[new_column_name] = df[column_name].apply(strip_and_convert)
    
    return df



In [16]:
numeric_cols = ['Clock Speed','RAM Size','Display Size','Screen Resolution','Laptop Weight','Warranty','Average Battery Life','Price']
for col in numeric_cols:
    df = convert_to_numeric(df,col)
df.head(21)


,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,...,Price,Description,Clock Speed (numeric),RAM Size (numeric),Display Size (numeric),Screen Resolution (numeric),Laptop Weight (numeric),Warranty (numeric),Average Battery Life (numeric),Price (numeric)
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,...,"35,000",The Dell Inspiron is a versatile laptop that c...,2.4,8.0,15.6,19201080.0,2.50,1.0,6.0,35000.0
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,...,"55,000",The MSI GL65 is a high-performance laptop desi...,2.6,16.0,15.6,19201080.0,2.30,2.0,4.0,55000.0
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,...,"90,000",The HP EliteBook is a premium laptop designed ...,2.8,16.0,14.0,19201080.0,1.50,3.0,8.0,90000.0
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,...,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,2.1,8.0,15.6,1366768.0,2.20,1.0,5.0,25000.0
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,...,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,3.1,64.0,15.6,38402160.0,1.80,2.0,7.0,200000.0
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,...,"80,000",The Acer Predator is a powerhouse laptop desig...,2.8,16.0,17.3,19201080.0,3.20,1.0,5.0,80000.0
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,...,"90,000",The Microsoft Surface Laptop is a premium devi...,1.6,8.0,13.5,22561504.0,1.30,1.0,10.0,90000.0
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,...,"60,000",The Lenovo ThinkPad is a powerful laptop desig...,3.0,16.0,14.0,25601440.0,1.60,3.0,6.0,60000.0
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,...,"30,000",The HP Pavilion is a budget-friendly laptop th...,2.3,12.0,15.6,1366768.0,2.10,1.0,4.0,30000.0
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,...,"85,000",The ASUS ROG Strix G is a high-performance gam...,2.9,16.0,17.3,19201080.0,2.90,2.0,5.0,85000.0


In [17]:
import pandas as pd

def rank_column(df, column_name, new_column_name):
    """
    Adds a new column to the DataFrame with rankings based on the specified column.
    Ranks are categorized as Low(1), Medium(2), and High(3) based on the values in the column.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The column to base the rankings on.
        new_column_name (str): The name of the new column for rankings.

    Returns:
        pd.DataFrame: The updated DataFrame with the new ranking column.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Calculate the quantiles
    low_threshold = df[column_name].quantile(0.33)
    high_threshold = df[column_name].quantile(0.66)

    # Apply ranking
    def categorize(value):
        if value <= low_threshold:
            return 1  # Low
        elif value <= high_threshold:
            return 2  # Medium
        else:
            return 3  # High

    # Create the new column
    df[new_column_name] = df[column_name].apply(categorize)

    return df




### Rank all the numeric columns into Low(1) , Medium(2) , High(3)
- this will help

In [18]:
numeric_cols = ['Clock Speed (numeric)','RAM Size (numeric)','Display Size (numeric)','Screen Resolution (numeric)','Laptop Weight (numeric)','Warranty (numeric)','Average Battery Life (numeric)','Price (numeric)']
for col in numeric_cols:
    df = rank_column(df,col,col+' Rank')
df.head(21)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,...,Average Battery Life (numeric),Price (numeric),Clock Speed (numeric) Rank,RAM Size (numeric) Rank,Display Size (numeric) Rank,Screen Resolution (numeric) Rank,Laptop Weight (numeric) Rank,Warranty (numeric) Rank,Average Battery Life (numeric) Rank,Price (numeric) Rank
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,...,6.0,35000.0,1,1,2,1,3,1,1,1
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,...,4.0,55000.0,1,1,2,1,3,2,1,1
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,...,8.0,90000.0,2,1,1,1,1,3,2,2
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,...,5.0,25000.0,1,1,2,1,3,1,1,1
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,...,7.0,200000.0,3,3,2,3,2,2,2,3
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,...,5.0,80000.0,2,1,3,1,3,1,1,2
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,...,10.0,90000.0,1,1,1,2,1,1,3,2
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,...,6.0,60000.0,3,1,1,3,2,3,1,1
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,...,4.0,30000.0,1,1,2,1,3,1,1,1
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,...,5.0,85000.0,3,1,3,1,3,2,1,2


### Get data as json from processed dataframe 

In [19]:
 laptop_columns = df.columns.tolist() 
laptop_columns

['Brand',
 'Model Name',
 'Core',
 'CPU Manufacturer',
 'Clock Speed',
 'RAM Size',
 'Storage Type',
 'Display Type',
 'Display Size',
 'Graphics Processor',
 'Screen Resolution',
 'OS',
 'Laptop Weight',
 'Special Features',
 'Warranty',
 'Average Battery Life',
 'Price',
 'Description',
 'Clock Speed (numeric)',
 'RAM Size (numeric)',
 'Display Size (numeric)',
 'Screen Resolution (numeric)',
 'Laptop Weight (numeric)',
 'Warranty (numeric)',
 'Average Battery Life (numeric)',
 'Price (numeric)',
 'Clock Speed (numeric) Rank',
 'RAM Size (numeric) Rank',
 'Display Size (numeric) Rank',
 'Screen Resolution (numeric) Rank',
 'Laptop Weight (numeric) Rank',
 'Warranty (numeric) Rank',
 'Average Battery Life (numeric) Rank',
 'Price (numeric) Rank']

In [20]:
import pandas as pd
import json

def get_laptops_in_json(df):
    """
    Extracts laptop details from a CSV file and returns them in JSON format.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        str: A JSON-formatted string of laptop details.
    """
    try:

        # Select relevant columns for laptop details
        laptop_columns = df.columns.tolist()  # Include all columns dynamically

        if not laptop_columns:
            raise ValueError("No columns found in the CSV file.")

        # Filter the DataFrame to include only the relevant columns
        laptops_df = df[laptop_columns]

        # Convert the DataFrame to JSON
        laptops_json = laptops_df.to_json(orient="records", indent=4)

        return laptops_json

    except Exception as e:
        return json.dumps({"error": str(e)})

In [21]:
json_output = get_laptops_in_json(df)
print(json_output)

[
    {
        "Brand":"Dell",
        "Model Name":"Inspiron",
        "Core":"i5",
        "CPU Manufacturer":"Intel",
        "Clock Speed":"2.4 GHz",
        "RAM Size":"8GB",
        "Storage Type":"SSD",
        "Display Type":"LCD",
        "Display Size":"15.6\"",
        "Graphics Processor":"Intel UHD",
        "Screen Resolution":"1920x1080",
        "OS":"Windows 10",
        "Laptop Weight":"2.5 kg",
        "Special Features":"Backlit Keyboard",
        "Warranty":"1 year",
        "Average Battery Life":"6 hours",
        "Price":"35,000",
        "Description":"The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6\" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewin

# Test Moderations API

In [7]:
from openai import OpenAI
client = OpenAI()

response = client.moderations.create(
    model="omni-moderation-latest",
    input="...text to classify goes here...",
)

print(response)

ModerationCreateResponse(id='modr-93d55d06d8d61a31e51c76e39189631d', model='omni-moderation-latest-intents', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=False, illicit_violent=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=False, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['

- input: "...text to classify goes here...",
- see that there is no harmful information in the input, hence it was ***not flaged***.

In [11]:
print(response.model_dump_json())

{"id":"modr-93d55d06d8d61a31e51c76e39189631d","model":"omni-moderation-latest-intents","results":[{"categories":{"harassment":false,"harassment_threatening":false,"hate":false,"hate_threatening":false,"illicit":false,"illicit_violent":false,"self_harm":false,"self_harm_instructions":false,"self_harm_intent":false,"sexual":false,"sexual_minors":false,"violence":false,"violence_graphic":false,"harassment/threatening":false,"hate/threatening":false,"illicit/violent":false,"self-harm/intent":false,"self-harm/instructions":false,"self-harm":false,"sexual/minors":false,"violence/graphic":false},"category_applied_input_types":{"harassment":["text"],"harassment_threatening":["text"],"hate":["text"],"hate_threatening":["text"],"illicit":["text"],"illicit_violent":["text"],"self_harm":["text"],"self_harm_instructions":["text"],"self_harm_intent":["text"],"sexual":["text"],"sexual_minors":["text"],"violence":["text"],"violence_graphic":["text"],"harassment/threatening":["text"],"hate/threatening"

In [13]:
from openai import OpenAI
client = OpenAI()

response = client.moderations.create(
    model="omni-moderation-latest",
    input="I need a right computer to HACK all the systems in the world.",
)

print(response.model_dump_json())

{"id":"modr-6679d4e090eda857dcb72bcbc3574a5a","model":"omni-moderation-latest-intents","results":[{"categories":{"harassment":false,"harassment_threatening":false,"hate":false,"hate_threatening":false,"illicit":true,"illicit_violent":false,"self_harm":false,"self_harm_instructions":false,"self_harm_intent":false,"sexual":false,"sexual_minors":false,"violence":false,"violence_graphic":false,"harassment/threatening":false,"hate/threatening":false,"illicit/violent":false,"self-harm/intent":false,"self-harm/instructions":false,"self-harm":false,"sexual/minors":false,"violence/graphic":false},"category_applied_input_types":{"harassment":["text"],"harassment_threatening":["text"],"hate":["text"],"hate_threatening":["text"],"illicit":["text"],"illicit_violent":["text"],"self_harm":["text"],"self_harm_instructions":["text"],"self_harm_intent":["text"],"sexual":["text"],"sexual_minors":["text"],"violence":["text"],"violence_graphic":["text"],"harassment/threatening":["text"],"hate/threatening":

- input := "I need a right computer to HACK all the systems in the world."
- see that there is harmful information in the input, hence it was ***flaged***.

In [105]:
good_response=json.loads('''{"id":"modr-93d55d06d8d61a31e51c76e39189631d","model":"omni-moderation-latest-intents","results":[{"categories":{"harassment":false,"harassment_threatening":false,"hate":false,"hate_threatening":false,"illicit":false,"illicit_violent":false,"self_harm":false,"self_harm_instructions":false,"self_harm_intent":false,"sexual":false,"sexual_minors":false,"violence":false,"violence_graphic":false,"harassment/threatening":false,"hate/threatening":false,"illicit/violent":false,"self-harm/intent":false,"self-harm/instructions":false,"self-harm":false,"sexual/minors":false,"violence/graphic":false},"category_applied_input_types":{"harassment":["text"],"harassment_threatening":["text"],"hate":["text"],"hate_threatening":["text"],"illicit":["text"],"illicit_violent":["text"],"self_harm":["text"],"self_harm_instructions":["text"],"self_harm_intent":["text"],"sexual":["text"],"sexual_minors":["text"],"violence":["text"],"violence_graphic":["text"],"harassment/threatening":["text"],"hate/threatening":["text"],"illicit/violent":["text"],"self-harm/intent":["text"],"self-harm/instructions":["text"],"self-harm":["text"],"sexual/minors":["text"],"violence/graphic":["text"]},"category_scores":{"harassment":0.000050333557545281144,"harassment_threatening":7.14190139989638e-6,"hate":0.000015598027633743823,"hate_threatening":9.368717268777097e-7,"illicit":0.000023413582477639402,"illicit_violent":0.000020027376671813928,"self_harm":2.2474089854291757e-6,"self_harm_instructions":1.0953122821495382e-6,"self_harm_intent":1.1843139025979655e-6,"sexual":0.00004305570813246149,"sexual_minors":0.00001141223854434603,"violence":0.000017400543552708932,"violence_graphic":6.10885510164857e-6,"harassment/threatening":7.14190139989638e-6,"hate/threatening":9.368717268777097e-7,"illicit/violent":0.000020027376671813928,"self-harm/intent":1.1843139025979655e-6,"self-harm/instructions":1.0953122821495382e-6,"self-harm":2.2474089854291757e-6,"sexual/minors":0.00001141223854434603,"violence/graphic":6.10885510164857e-6},"flagged":false}]}''')
bad_response=json.loads('''{"id":"modr-6679d4e090eda857dcb72bcbc3574a5a","model":"omni-moderation-latest-intents","results":[{"categories":{"harassment":false,"harassment_threatening":false,"hate":false,"hate_threatening":false,"illicit":true,"illicit_violent":false,"self_harm":false,"self_harm_instructions":false,"self_harm_intent":false,"sexual":false,"sexual_minors":false,"violence":false,"violence_graphic":false,"harassment/threatening":false,"hate/threatening":false,"illicit/violent":false,"self-harm/intent":false,"self-harm/instructions":false,"self-harm":false,"sexual/minors":false,"violence/graphic":false},"category_applied_input_types":{"harassment":["text"],"harassment_threatening":["text"],"hate":["text"],"hate_threatening":["text"],"illicit":["text"],"illicit_violent":["text"],"self_harm":["text"],"self_harm_instructions":["text"],"self_harm_intent":["text"],"sexual":["text"],"sexual_minors":["text"],"violence":["text"],"violence_graphic":["text"],"harassment/threatening":["text"],"hate/threatening":["text"],"illicit/violent":["text"],"self-harm/intent":["text"],"self-harm/instructions":["text"],"self-harm":["text"],"sexual/minors":["text"],"violence/graphic":["text"]},"category_scores":{"harassment":0.005059632854340465,"harassment_threatening":0.0009637153569543876,"hate":0.00008040859356292355,"hate_threatening":0.000021654404989510328,"illicit":0.6845479055493316,"illicit_violent":0.00011335690050875118,"self_harm":0.000014653129390717958,"self_harm_instructions":9.818326983657703e-7,"self_harm_intent":0.00021250413756646795,"sexual":0.00001535623976496012,"sexual_minors":1.9525885208642222e-6,"violence":0.0012958875223580482,"violence_graphic":7.722191834067996e-6,"harassment/threatening":0.0009637153569543876,"hate/threatening":0.000021654404989510328,"illicit/violent":0.00011335690050875118,"self-harm/intent":0.00021250413756646795,"self-harm/instructions":9.818326983657703e-7,"self-harm":0.000014653129390717958,"sexual/minors":1.9525885208642222e-6,"violence/graphic":7.722191834067996e-6},"flagged":true}]}''')


In [141]:
 good_response["results"][0]["flagged"]

False

In [143]:
 bad_response["results"][0]["flagged"]

True

In [157]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = client.moderations.create(model="omni-moderation-latest",input=user_input)
    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    print(moderation_output)
    # Check if the input was flagged by the moderation system.
    if moderation_output == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [159]:
moderation_check('completed full test, I want to hack all the systems in the world')

True


'Flagged'

# initilize conversation

In [38]:
def initialize_conversation():
    '''
    Initializes a conversation for an intelligent laptop gadget expert system.

    Returns:
        list: [{"role": "system", "content": system_message}]
    '''
    # System message defining the purpose and structure of the conversation
    system_message = (
        "{\"role\": \"\", \"content\": \"Hello! I'm your AI ShopAssist Chatbot, I am here to help you find the perfect laptop based on your needs.\"}"
        "Hello! I'm your AI ShopAssist Chatbot, I am here to help you find the perfect laptop based on your needs.\n\n"
        "**Purpose:**\n"
        "To understand your specific requirements and recommend the most suitable laptop options.\n\n"
        "**How to Proceed:**\n"
        "1. Share details about your laptop requirements, such as:\n"
        "   - GPU intensity: (e.g., high for gaming, moderate for design, or low for basic use).\n"
        "   - Display quality: (e.g., Full HD, 4K, or focus on color accuracy).\n"
        "   - Portability: (e.g., lightweight, medium, or no preference).\n"
        "   - Multitasking needs: (e.g., casual use, professional work, or heavy tasks).\n"
        "   - Processing speed: (e.g., basic, fast, or extremely fast).\n"
        "   - Budget: (e.g., $500-$1000).\n"
        "2. I'll ask follow-up questions if needed to fill in the details.\n"
        "3. Once we finalize your profile, I'll confirm it with you before recommending laptops.\n\n"
        "**Example Conversation:**\n"
        "User: I need a laptop for gaming and graphic design.\n"
        "System: Got it! I'll ensure a high-end GPU and a color-accurate display. What's your budget range?\n\n"
        "User: My budget is $1500.\n"
        "System: Understood! I'll prioritize performance and display quality within your budget. Do you need portability or additional features like touch screens?\n\n"
        "**How This Works:**\n"
        "1. Thought 1: Gather and confidently fill in details for your profile.\n"
        "2. Thought 2: Use follow-ups to refine and complete missing details.\n"
        "3. Thought 3: Confirm and verify the profile before making recommendations.\n\n"
        "Let's get started! Tell me about your laptop requirements."
    )

    # Return the initial message as a list in the required structure
    return [{"role": "system", "content": system_message}]



In [9]:
input_messages = initialize_conversation()
for message in input_messages:
    print(f"{message['role'].capitalize()}: {message['content']}")


System: Hello! I'm your AI ShopAssist Chatbot, I am here to help you find the perfect laptop based on your needs.

**Purpose:**
To understand your specific requirements and recommend the most suitable laptop options.

**How to Proceed:**
1. Share details about your laptop requirements, such as:
   - GPU intensity: (e.g., high for gaming, moderate for design, or low for basic use).
   - Display quality: (e.g., Full HD, 4K, or focus on color accuracy).
   - Portability: (e.g., lightweight, medium, or no preference).
   - Multitasking needs: (e.g., casual use, professional work, or heavy tasks).
   - Processing speed: (e.g., basic, fast, or extremely fast).
   - Budget: (e.g., $500-$1000).
2. I'll ask follow-up questions if needed to fill in the details.
3. Once we finalize your profile, I'll confirm it with you before recommending laptops.

**Example Conversation:**
User: I need a laptop for gaming and graphic design.
System: Got it! I'll ensure a high-end GPU and a color-accurate displa

# Chat Completions

In [40]:

input_msg = [
    {"role": "user", "content": "I need a laptop for watching Netflix, listening to music and browse internet"}
]

In [32]:
messages = initialize_conversation()
    
    # Add user-provided messages
messages.extend(input_msg)
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'system',
  'content': "Hello! I'm your AI ShopAssist Chatbot, I am here to help you find the perfect laptop based on your needs."},
 {'role': 'user',
  'content': 'I need a laptop for watching Netflix, listening to music and browse internet'}]

In [161]:
# get_chat_completions function - this function perform LLM call to get LLM Response
# retry upto 6 attempts,waiting time min 1sec to maxing 20sec
 # Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'
    #this is done in the conversation dialog - it will be present in the context history.
    #Narayana messages = initialize_conversation()
    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output
        # Add user-provided messages
        messages.extend(input)
        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)
        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        messages.extend(input)
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)
        output = chat_completion.choices[0].message.content

    return output

In [52]:
try:
    # Plain text response
    text_response = get_chat_completions(input_msg, json_format=True)
    print("JSON Response:", json.dumps(text_response, indent=4))

except Exception as e:
    print("Error:", e)


JSON Response: {
    "output": {
        "laptop": "For watching Netflix, listening to music, and browsing the internet, consider a laptop with at least a Full HD display, decent speakers, and a fast internet connection. Some recommended laptops include the Lenovo Yoga C740, Dell XPS 13, and MacBook Air."
    }
}


In [54]:
text_response

{'output': {'laptop': 'For watching Netflix, listening to music, and browsing the internet, consider a laptop with at least a Full HD display, decent speakers, and a fast internet connection. Some recommended laptops include the Lenovo Yoga C740, Dell XPS 13, and MacBook Air.'}}

### intent_confirmation_layer()

In [166]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    allowed_values = {'low','medium','high'}

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

### dictionary_present()

In [173]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [ ]:
def evaluate_model_response(tagged_dict, model_dict):
  score = 0
  mappings = {
      'low': 0,
      'medium': 1,
      'high': 2
  }

  for key in tagged_dict.keys():
    if key == 'budget':
      continue
    tagged_value = tagged_dict[key]
    model_value = model_dict[key]
    tagged_mapping = mappings.get(tagged_value, -1)
    model_mapping = mappings.get(model_value, -1)

    if model_mapping >= tagged_mapping:
        score += 1

  return score


# Integrating the Function Calling API
- Validate if all required inputs are collected from User
- Use to connect to database to get the inventory of available laptops
- Perform matching and scoring among the selected laptops and select top 3 for the user/customer.

## Testing it all together, in Python Jyputer Notebook


In [181]:
def dialogue_mgmt_system():
    """
    Manages a conversational system between a user and an assistant to recommend laptops.

    This function initializes the conversation, retrieves the introduction, handles user input,
    performs moderation checks, and manages the recommendation process based on user interactions.

    Returns:
        None
    """

    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')

    top_3_laptops = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        #Narayana for testing: moderation = moderation_check(user_input)
        moderation='Narayana'
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            #Narayana for testing: moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')

                response = dictionary_present(response_assistant)
                print(response)
                '''
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)

                print("top 3 laptops are", top_3_laptops)

                validated_reco = recommendation_validation(top_3_laptops)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n
                '''
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            #Narayana for testing: moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [183]:
dialogue_mgmt_system()

"I'm here to assist you in finding the perfect laptop based on your needs. What specific requirements do you have for your new laptop? Feel free to share details like GPU intensity, display quality, portability, multitasking needs, processing speed, and budget.\n"

 I want a laptop to browse internet , watch netflix , listen to songs


Intent Confirmation Yes/No: No

Great! For your needs of internet browsing, streaming Netflix, and listening to music, you don't require high-end specifications like a dedicated GPU or ultra-fast processing speed. Here are a few key elements to consider:

- Display Quality: A good quality Full HD display would be ideal for enjoying Netflix content and music videos.
- Portability: Since you might want to move around with your laptop, a lightweight or medium-weight laptop would be convenient for you.
- Battery Life: Considering you'll be streaming content and listening to music, a laptop with a decent battery life would be beneficial.
- Storage: Sufficient storage for saving music files and offline Netflix content might be useful.
- Budget: Do you have a specific budget range in mind for this laptop?

Once we have your budget range, we can select a laptop that best fits your needs. Feel free to provide any additional preferences you may have!



 90000


Intent Confirmation Yes/No: No

Great, thank you for providing your budget of $90,000. With that budget, we can definitely find you a laptop that meets your needs for internet browsing, streaming Netflix, and listening to music.

Considering your budget and requirements, I would recommend a high-quality laptop with the following features:

- Full HD display for an optimal viewing experience of Netflix and music videos.
- Good battery life to ensure you can enjoy your entertainment without frequent charging.
- Fast and responsive performance for smooth internet browsing and multitasking.
- Adequate storage space to store music, videos, and other files.

Based on these considerations, we can look into premium laptops with top-tier specifications. Would you like me to focus on any specific brand preferences, or are there any other features you would like to prioritize within this budget? Let me know so we can narrow down the options further.



...

 yes, Thank you


Intent Confirmation Yes/No: No

You're welcome! If you have any more questions in the future or need further assistance in choosing your laptop or any other tech-related queries, feel free to reach out. Enjoy your new Dell Latitude laptop, and have a fantastic browsing, streaming, and music experience ahead! Have a great day!



 - GPU intensity: high - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 80,000 INR


Intent Confirmation Yes/No: Yes

Thank you for providing your detailed laptop requirements. Based on your preferences, here's a summary of what we are looking for:

- **GPU Intensity:** High
- **Display Quality:** High
- **Portability:** Low
- **Multitasking:** High
- **Processing Speed:** Medium
- **Budget:** 80,000 INR

Considering these specifications, we would aim for a laptop with a powerful GPU for intensive tasks, high-quality display for enhanced visuals, sturdy build for low portability needs, good multitasking capabilities, and decent processing speed within your budget of 80,000 INR.

I'll look into options that match your requirements and provide recommendations accordingly to help you find the ideal laptop. Let's explore models that align with your needs.


Variables extracted!

{'GPU intensity': 'High', 'Display quality': 'High', 'Portability': 'Low', 'Multitasking': 'High', 'Processing speed': 'Medium', 'Budget': '80000'}


KeyboardInterrupt: Interrupted by user

### Improve the way we score the laptops to identify perfect/possible match - this can be further refined.
- picking the user requirements from above chat completions conversation
- {'GPU intensity': 'High', 'Display quality': 'High', 'Portability': 'Low', 'Multitasking': 'High', 'Processing speed': 'Medium', 'Budget': '80000'}


In [13]:

user_requirements = json.loads("""
{
    "GPU intensity": "High",
    "Display quality": "High",
    "Portability": "Low",
    "Multitasking": "High",
    "Processing speed": "Medium",
    "Budget": "80000"
}
""")


In [22]:
def compare_laptops_with_user(user_requirements,df):
    
    def map_priority(value):
        """Map High, Medium, Low to numeric values."""
        priority_map = {"High": 3, "Medium": 2, "Low": 1}
        return priority_map.get(value, 0)

    def calculate_score(laptop, user_values):
        """Calculate a matching score for a laptop based on user preferences."""
        score = 0

        # GPU matching (mapped to CPU core rank as proxy)
        if map_priority(user_values.get("GPU intensity")) <= laptop["Clock Speed (numeric) Rank"]:
            score += 1

        # Display quality matching
        if map_priority(user_values.get("Display quality")) <= min(
            laptop["Display Size (numeric) Rank"],
            laptop["Screen Resolution (numeric) Rank"]
        ):
            score += 1

        # Portability matching
        if map_priority(user_values.get("Portability")) <= max(
            laptop["Laptop Weight (numeric) Rank"],
            laptop["Average Battery Life (numeric) Rank"]
        ):
            score += 1

        # Multitasking (mapped to RAM size rank)
        if map_priority(user_values.get("Multitasking")) <= laptop["RAM Size (numeric) Rank"]:
            score += 1

        # Processing speed (mapped to clock speed rank)
        if map_priority(user_values.get("Processing speed")) <= laptop["Clock Speed (numeric) Rank"]:
            score += 1

        # Budget matching
        if float(user_values.get("Budget", 0)) >= laptop["Price (numeric)"]:
            score += 1

        return score

    laptops = get_laptops_in_json(df)
    user_values = {key: map_priority(value) for key, value in user_requirements.items()}

    # Score each laptop and sort by score (descending), breaking ties by price (ascending)
    scored_laptops = [
        {
            "laptop": laptop,
            "score": calculate_score(laptop, user_values)
        }
        for laptop in laptops
    ]

    sorted_laptops = sorted(
        scored_laptops,
        key=lambda x: (-x["score"], x["laptop"]["Price (numeric)"])
    )

    # Filter laptops by a minimum score of 4
    filtered_laptops = [entry for entry in sorted_laptops if entry["score"] >= 4]

    if not filtered_laptops:
        return "No possible matches - {}"

    # Return the top 3 matching laptops with scores
    top_matches = [
        {"laptop": entry["laptop"], "score": entry["score"]}
        for entry in filtered_laptops[:3]
    ]
    return f"Here are the top-3 matches: {json.dumps(top_matches, indent=2)}"


In [23]:
compare_laptops_with_user(user_requirements,df)

TypeError: string indices must be integers, not 'str'

# with Function calling

In [7]:
from typing import Dict, Any
import json
import openai

def dictionary_present(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Uses OpenAI's ChatGPT function calling API to verify the presence of required keys in the dictionary.
    """
    # Define the schema for validation
    schema = {
        "type": "object",
        "properties": {
            "GPU intensity": {"type": "string"},
            "Display quality": {"type": "string"},
            "Portability": {"type": "string"},
            "Multitasking": {"type": "string"},
            "Processing speed": {"type": "string"},
            "Budget": {"type": "string"}
        },
        "required": ["GPU intensity", "Display quality", "Portability", "Multitasking", "Processing speed", "Budget"]
    }

    # Simulate an API call to validate the input using function calling
    try:
        response = openai.chat.completions.create(
            #model="gpt-3.5-turbo-0613",
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a validator for user inputs."},
                {"role": "user", "content": json.dumps(data)}
            ],
            response_format = { "type": "json_object"},
            functions=[
                {
                    "name": "dictionary_present",
                    "description": "Validates the presence of required keys in the user dictionary.",
                    "parameters": schema
                }
            ],
            function_call={"name": "dictionary_present"}
        )

        # Extract the validated arguments from the response
        validated_data = response["choices"][0]["message"]["function_call"]["arguments"]
        return json.loads(validated_data)

    except Exception as e:
        raise ValueError(f"Validation failed: {str(e)}")

In [ ]:
dictionary_present()

In [8]:
# Function to compare laptops with user preferences
def compare_laptops_with_user(user_requirements):
    def map_priority(value):
        return {"High": 3, "Medium": 2, "Low": 1}.get(value, 0)

    def calculate_score(laptop, user_values):
        score = 0
        if map_priority(user_values.get("GPU intensity")) <= laptop["Clock Speed (numeric) Rank"]:
            score += 1
        if map_priority(user_values.get("Display quality")) <= min(
            laptop["Display Size (numeric) Rank"],
            laptop["Screen Resolution (numeric) Rank"]
        ):
            score += 1
        if map_priority(user_values.get("Portability")) <= max(
            laptop["Laptop Weight (numeric) Rank"],
            laptop["Average Battery Life (numeric) Rank"]
        ):
            score += 1
        if map_priority(user_values.get("Multitasking")) <= laptop["RAM Size (numeric) Rank"]:
            score += 1
        if map_priority(user_values.get("Processing speed")) <= laptop["Clock Speed (numeric) Rank"]:
            score += 1
        if float(user_values.get("Budget", 0)) >= laptop["Price (numeric)"]:
            score += 1
        return score

    laptops = get_laptops_in_json()
    user_values = {key: map_priority(value) for key, value in user_requirements.items()}

    scored_laptops = [
        {"laptop": laptop, "score": calculate_score(laptop, user_values)}
        for laptop in laptops
    ]

    sorted_laptops = sorted(
        scored_laptops,
        key=lambda x: (-x["score"], x["laptop"]["Price (numeric)"])
    )

    filtered_laptops = [entry for entry in sorted_laptops if entry["score"] >= 4]

    if not filtered_laptops:
        return "No possible matches - {}"

    return [
        {"laptop": entry["laptop"], "score": entry["score"]}
        for entry in filtered_laptops[:3]
    ]

# Integrating the function call into ChatCompletion
response = openai.chat.completions.create(
            #model="gpt-3.5-turbo-0613",
            model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a laptop recommendation assistant."},
        {"role": "user", "content": "Find the best laptops based on my requirements."}
    ],
    functions=[
        {
            "name": "compare_laptops_with_user",
            "description": "Recommend top laptops based on user requirements.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_requirements": {
                        "type": "object",
                        "properties": {
                            "GPU intensity": {"type": "string"},
                            "Display quality": {"type": "string"},
                            "Portability": {"type": "string"},
                            "Multitasking": {"type": "string"},
                            "Processing speed": {"type": "string"},
                            "Budget": {"type": "string"}
                        },
                        "required": ["GPU intensity", "Display quality", "Portability", "Multitasking", "Processing speed", "Budget"]
                    }
                },
                "required": ["user_requirements"]
            }
        }
    ],
    function_call={
        "name": "compare_laptops_with_user",
        "arguments": json.dumps({
            "user_requirements": {
                "GPU intensity": "High",
                "Display quality": "High",
                "Portability": "Low",
                "Multitasking": "High",
                "Processing speed": "Medium",
                "Budget": "80000"
            }
        })
    }
)

# Process and print the API response
function_response = response["choices"][0]["message"]["function_call"]["arguments"]
print("Top laptop recommendations:", json.dumps(json.loads(function_response), indent=2))

TypeError: 'ChatCompletion' object is not subscriptable

In [9]:
response

ChatCompletion(id='chatcmpl-AfD355Ocv45wtgS1NRAtttCO03ar7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"user_requirements":{"GPU intensity":"High","Display quality":"4k","Portability":"Lightweight","Multitasking":"Intensive","Processing speed":"Fast","Budget":"2000"}}', name='compare_laptops_with_user'), tool_calls=None))], created=1734384971, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=38, prompt_tokens=108, total_tokens=146, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

# Improvements to Overall Application
- Improve the inventory mix, we only have 21 products in the inventory
- Create a mapping of the 6 parameters to assosiate each of the attributes like shown below
* GPI(Core)
* Display quality - (Display Type, Display Type, screen resolution, Graphics Processoe)
* Portability (Laptop weight, Average Battery Life)
* Multitasking
* Processing speed (Clock Speed, RAM size)
* budget(Price, Warrenty)